## Why Data Splitting Matters

In supervised machine learning, the goal is to build a model that learns how to correctly connect inputs to outputs. The inputs are often called **features** or **predictors**, while the outputs are known as **targets** or **responses**.

How well a model performs depends on the type of problem you’re solving.  
- For **regression** tasks, performance is usually measured using metrics like **R²**, **root mean square error (RMSE)**, or **mean absolute error (MAE)**.  
- For **classification** problems, common metrics include **accuracy**, **precision**, **recall**, and the **F1 score**.

There’s no single *perfect* value for these metrics—what’s considered good performance can vary widely depending on the industry or use case. While many resources explain these metrics in detail, the most important thing to remember is this:

> **A model must be evaluated fairly to be trusted.**

You can’t accurately judge a model’s performance using the same data it was trained on, because the model has already *seen* that data. This would result in overly optimistic performance scores. Instead, the model should be tested on **new, unseen data** to understand how well it will perform in real-world situations.

That’s why we split our dataset before training. One part is used to **train** the model, and another part is kept aside to **test** it. This separation ensures that performance metrics reflect the model’s true predictive ability, not just its ability to memorize the training data.


## Training, Validation, and Test Sets

Splitting your dataset is a key step in making sure your model’s performance is evaluated fairly and realistically. In most machine learning projects, the dataset is randomly divided into **three parts**:

### Training Set
The **training set** is used to teach the model. This is where the model learns patterns in the data by adjusting its internal parameters.  
For example, during training, a model learns the best weights or coefficients in algorithms like **linear regression**, **logistic regression**, or **neural networks**.

### Validation Set
The **validation set** is used to evaluate the model while you are fine-tuning it. This is especially useful during **hyperparameter tuning**.  
For instance, if you’re deciding how many neurons to use in a neural network or which kernel works best for a support vector machine, you try different options. For each option, the model is trained on the training set and evaluated using the validation set to see which configuration performs best.

### Test Set
The **test set** is reserved for the final evaluation of the model. It provides an unbiased measure of how well the model performs on completely new data.  
This dataset should **not** be used during training or validation, as doing so would compromise the fairness of the evaluation.

In simpler scenarios—where hyperparameter tuning isn’t needed—it’s often acceptable to work with just **training** and **test** sets.

---

## Underfitting and Overfitting

Splitting data also helps identify two common modeling problems: **underfitting** and **overfitting**.

### Underfitting
**Underfitting** occurs when a model is too simple to capture the underlying patterns in the data.  
For example, using a linear model to describe a clearly nonlinear relationship can lead to underfitting. These models usually perform poorly on both the training and test datasets because they fail to learn meaningful patterns.

### Overfitting
**Overfitting** happens when a model is too complex and learns not only the true patterns in the data but also the noise.  
Such models often perform extremely well on the training data but fail to generalize to new, unseen data. As a result, their performance on the test set is usually much worse.

---

Splitting your dataset properly helps you balance learning and generalization, ensuring that your model performs well not just on known data, but also in real-world scenarios.


In [9]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    train_test_split,
    KFold,
    StratifiedKFold,
    RepeatedKFold,
    LeaveOneOut,
    ShuffleSplit,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
    GroupKFold,
    StratifiedGroupKFold,
    PredefinedSplit,
    cross_val_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)



In [10]:


df = pd.read_csv('student_performance_updated_1000.csv')

In [11]:
df.shape

(1000, 12)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   StudentID                  960 non-null    float64
 1   Name                       966 non-null    object 
 2   Gender                     952 non-null    object 
 3   AttendanceRate             960 non-null    float64
 4   StudyHoursPerWeek          950 non-null    float64
 5   PreviousGrade              967 non-null    float64
 6   ExtracurricularActivities  957 non-null    float64
 7   ParentalSupport            978 non-null    object 
 8   FinalGrade                 960 non-null    float64
 9   Study Hours                976 non-null    float64
 10  Attendance (%)             959 non-null    float64
 11  Online Classes Taken       975 non-null    object 
dtypes: float64(8), object(4)
memory usage: 93.9+ KB


In [13]:
df.head()

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Attendance (%),Online Classes Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
4,5.0,Emma,Female,NaN,18.0,82.0,2.0,Medium,85.0,4.1,97.0,True


In [14]:
df.isnull().sum()

StudentID                    40
Name                         34
Gender                       48
AttendanceRate               40
StudyHoursPerWeek            50
PreviousGrade                33
ExtracurricularActivities    43
ParentalSupport              22
FinalGrade                   40
Study Hours                  24
Attendance (%)               41
Online Classes Taken         25
dtype: int64

In [15]:
# Prepare data for machine learning

# Create binary target: Pass (1) if FinalGrade >= 70, else Fail (0)
df_clean = df.dropna(subset=['FinalGrade'])
df_clean['Pass'] = (df_clean['FinalGrade'] >= 70).astype(int)

# Select features
feature_cols = ['AttendanceRate', 'StudyHoursPerWeek', 'PreviousGrade', 
                'ExtracurricularActivities', 'Study Hours', 'Attendance (%)']

# Clean data
df_clean = df_clean.dropna(subset=feature_cols)

# Prepare X and y
X = df_clean[feature_cols].values
y = df_clean['Pass'].values

# Add Gender encoding
if 'Gender' in df_clean.columns:
    le = LabelEncoder()
    gender_encoded = le.fit_transform(df_clean['Gender'].fillna('Unknown'))
    X = np.column_stack([X, gender_encoded])



In [21]:
print(f"Features shape: {X.shape}")


Features shape: (767, 7)


In [22]:
print(f"Target shape: {y.shape}")


Target shape: (767,)


In [23]:
print(f"Class Distribution:")
print(f"  Pass (1): {sum(y)} samples ({sum(y)/len(y)*100:.2f}%)")
print(f"  Fail (0): {len(y)-sum(y)} samples ({(len(y)-sum(y))/len(y)*100:.2f}%)")

Class Distribution:
  Pass (1): 615 samples (80.18%)
  Fail (0): 152 samples (19.82%)


### 1. Simple Train-Test Split

**What it does:** Randomly divides data into training and testing sets.

**When to use:**
- Quick model evaluation
- Large datasets
- Initial prototyping

**Parameters:**
- `test_size`: Proportion of dataset for testing (e.g., 0.2 = 20%)
- `random_state`: Seed for reproducibility
- `stratify`: Maintains class distribution

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [26]:
print(f"Total samples: {len(X)}")
print(f"\nTraining set:")
print(f"  Size: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Pass: {sum(y_train)} ({sum(y_train)/len(y_train)*100:.1f}%)")
print(f"  Fail: {len(y_train)-sum(y_train)} ({(len(y_train)-sum(y_train))/len(y_train)*100:.1f}%)")


Total samples: 767

Training set:
  Size: 613 samples (79.9%)
  Pass: 492 (80.3%)
  Fail: 121 (19.7%)


In [27]:
print(f"\nTesting set:")
print(f"  Size: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"  Pass: {sum(y_test)} ({sum(y_test)/len(y_test)*100:.1f}%)")
print(f"  Fail: {len(y_test)-sum(y_test)} ({(len(y_test)-sum(y_test))/len(y_test)*100:.1f}%)")



Testing set:
  Size: 154 samples (20.1%)
  Pass: 123 (79.9%)
  Fail: 31 (20.1%)


### 2. Train-Validation-Test Split (3-Way Split)

**What it does:** Divides data into THREE sets:
- **Training:** To train the model
- **Validation:** To tune hyperparameters
- **Test:** For final evaluation

**When to use:**
- Hyperparameter tuning
- Model selection
- When you need unbiased final evaluation

In [29]:
# First split: separate test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [30]:
# Second split: separate train and validation (60-20)
# From remaining 80%, we want 60% train and 20% validation
# So validation is 20/80 = 0.25 of the temp set
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

In [31]:
print(f"Total samples: {len(X)}")
print(f"\nTraining set:   {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation set: {len(X_val)} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"Testing set:    {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nVerification: {len(X_train)} + {len(X_val)} + {len(X_test)} = {len(X_train) + len(X_val) + len(X_test)} samples")


Total samples: 767

Training set:   459 samples (59.8%)
Validation set: 154 samples (20.1%)
Testing set:    154 samples (20.1%)

Verification: 459 + 154 + 154 = 767 samples


## 3. Cross Validation

### 3.1 K-Fold Cross-Validation

**What it does:** Divides data into K equal folds, trains on K-1 folds, tests on 1 fold, repeats K times.

**When to use:**
- Standard evaluation method
- Medium to large datasets
- Need robust performance estimate

**Common K values:**
- K=3: Quick evaluation
- K=5: Standard choice
- K=10: Thorough evaluation

In [32]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


In [33]:
print(f"Total samples: {len(X)}")
print(f"Number of folds: 5")
print(f"\nSplit details:")

for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(X), 1):
    print(f"\n  Fold {fold_idx}:")
    print(f"    Training: {len(train_idx)} samples (indices {train_idx[0]}-{train_idx[-1]})")
    print(f"    Testing:  {len(test_idx)} samples (indices {test_idx[0]}-{test_idx[-1]})")
    print(f"    Test Pass Rate: {y[test_idx].mean()*100:.1f}%")


Total samples: 767
Number of folds: 5

Split details:

  Fold 1:
    Training: 613 samples (indices 0-766)
    Testing:  154 samples (indices 2-763)
    Test Pass Rate: 83.8%

  Fold 2:
    Training: 613 samples (indices 0-766)
    Testing:  154 samples (indices 6-759)
    Test Pass Rate: 83.1%

  Fold 3:
    Training: 614 samples (indices 1-764)
    Testing:  153 samples (indices 0-766)
    Test Pass Rate: 83.0%

  Fold 4:
    Training: 614 samples (indices 0-766)
    Testing:  153 samples (indices 3-752)
    Test Pass Rate: 74.5%

  Fold 5:
    Training: 614 samples (indices 0-766)
    Testing:  153 samples (indices 1-764)
    Test Pass Rate: 76.5%


### 3.2 Stratified K-Fold Cross-Validation

**What it does:** K-Fold that preserves the class distribution in each fold.

**When to use:**
- **Imbalanced datasets** (IMPORTANT!)
- Classification problems
- When class distribution matters

**Why it matters:** Our dataset has 80% pass rate. Regular K-Fold might create folds with 70% or 90% pass rate. Stratified K-Fold maintains ~80% in ALL folds!

In [34]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [35]:
print(f"Total samples: {len(X)}")
print(f"Number of folds: 5")
print(f"\nSplit details (notice consistent pass rates):")

for fold_idx, (train_idx, test_idx) in enumerate(stratified_kfold.split(X, y), 1):
    train_pass_rate = y[train_idx].mean() * 100
    test_pass_rate = y[test_idx].mean() * 100
    
    print(f"\n  Fold {fold_idx}:")
    print(f"    Training: {len(train_idx)} samples, Pass Rate: {train_pass_rate:.1f}%")
    print(f"    Testing:  {len(test_idx)} samples, Pass Rate: {test_pass_rate:.1f}%")


Total samples: 767
Number of folds: 5

Split details (notice consistent pass rates):

  Fold 1:
    Training: 613 samples, Pass Rate: 80.3%
    Testing:  154 samples, Pass Rate: 79.9%

  Fold 2:
    Training: 613 samples, Pass Rate: 80.3%
    Testing:  154 samples, Pass Rate: 79.9%

  Fold 3:
    Training: 614 samples, Pass Rate: 80.1%
    Testing:  153 samples, Pass Rate: 80.4%

  Fold 4:
    Training: 614 samples, Pass Rate: 80.1%
    Testing:  153 samples, Pass Rate: 80.4%

  Fold 5:
    Training: 614 samples, Pass Rate: 80.1%
    Testing:  153 samples, Pass Rate: 80.4%


### 3.3 Repeated K-Fold Cross-Validation

**What it does:** Repeats K-Fold CV multiple times with different random splits.

**When to use:**
- Need very robust evaluation
- Critical applications
- Reduce impact of random variations

**Computational cost:** K × n_repeats (e.g., 5 folds × 3 repeats = 15 iterations)

In [37]:
# Example: 5-Fold with 3 Repeats

n_splits = 5
n_repeats = 3

repeated_kfold = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)


In [38]:
print(f"Total samples: {len(X)}")
print(f"Number of folds per repeat: {n_splits}")
print(f"Number of repeats: {n_repeats}")
print(f"Total iterations: {n_splits * n_repeats}")


Total samples: 767
Number of folds per repeat: 5
Number of repeats: 3
Total iterations: 15


In [39]:
iteration = 1
for repeat in range(n_repeats):
    print(f"\n  Repeat {repeat + 1}:")
    print(f"  {'-' * 70}")
    
    for fold in range(n_splits):
        train_idx, test_idx = list(repeated_kfold.split(X))[iteration - 1]
        print(f"    Iteration {iteration:2d} (Fold {fold + 1}): "
              f"Train={len(train_idx):3d} samples, Test={len(test_idx):3d} samples")
        iteration += 1



  Repeat 1:
  ----------------------------------------------------------------------
    Iteration  1 (Fold 1): Train=613 samples, Test=154 samples
    Iteration  2 (Fold 2): Train=613 samples, Test=154 samples
    Iteration  3 (Fold 3): Train=614 samples, Test=153 samples
    Iteration  4 (Fold 4): Train=614 samples, Test=153 samples
    Iteration  5 (Fold 5): Train=614 samples, Test=153 samples

  Repeat 2:
  ----------------------------------------------------------------------
    Iteration  6 (Fold 1): Train=613 samples, Test=154 samples
    Iteration  7 (Fold 2): Train=613 samples, Test=154 samples
    Iteration  8 (Fold 3): Train=614 samples, Test=153 samples
    Iteration  9 (Fold 4): Train=614 samples, Test=153 samples
    Iteration 10 (Fold 5): Train=614 samples, Test=153 samples

  Repeat 3:
  ----------------------------------------------------------------------
    Iteration 11 (Fold 1): Train=613 samples, Test=154 samples
    Iteration 12 (Fold 2): Train=613 samples, Tes

### 3.4 Leave-One-Out Cross-Validation (LOOCV)

**What it does:** Uses each sample as a test set once. For n samples, performs n iterations!

**When to use:**
- **Very small datasets** (<100 samples)
- Maximum data utilization needed
- Deterministic results required

**WARNING:** Extremely expensive! Our full dataset (767 samples) would need 767 iterations!

In [41]:
# Demonstration on a small subset
subset_size = 20
X_subset = X[:subset_size]
y_subset = y[:subset_size]


In [42]:
loo = LeaveOneOut()
n_splits_loo = loo.get_n_splits(X_subset)

In [43]:
print(f"\nSubset size: {subset_size} samples")
print(f"Number of iterations: {n_splits_loo}")
print(f"Training size per iteration: {subset_size - 1} samples")
print(f"Test size per iteration: 1 sample")



Subset size: 20 samples
Number of iterations: 20
Training size per iteration: 19 samples
Test size per iteration: 1 sample


In [44]:
print(f"\nFirst 10 iterations:")
for iteration, (train_idx, test_idx) in enumerate(loo.split(X_subset), 1):
    if iteration <= 10:
        print(f"  Iteration {iteration:2d}: Train on {len(train_idx)} samples, "
              f"Test on sample #{test_idx[0]} (class={y_subset[test_idx[0]]})")
    if iteration == 10:
        print(f"  ... ({subset_size - 10} more iterations)")


First 10 iterations:
  Iteration  1: Train on 19 samples, Test on sample #0 (class=1)
  Iteration  2: Train on 19 samples, Test on sample #1 (class=1)
  Iteration  3: Train on 19 samples, Test on sample #2 (class=0)
  Iteration  4: Train on 19 samples, Test on sample #3 (class=1)
  Iteration  5: Train on 19 samples, Test on sample #4 (class=0)
  Iteration  6: Train on 19 samples, Test on sample #5 (class=1)
  Iteration  7: Train on 19 samples, Test on sample #6 (class=1)
  Iteration  8: Train on 19 samples, Test on sample #7 (class=1)
  Iteration  9: Train on 19 samples, Test on sample #8 (class=0)
  Iteration 10: Train on 19 samples, Test on sample #9 (class=1)
  ... (10 more iterations)


### 3.5 Shuffle Split Cross-Validation

**What it does:** Creates random independent train/test splits.

**When to use:**
- Need custom train/test sizes
- Want specific number of iterations
- Flexible evaluation needed

**Difference from K-Fold:** Splits are independent - samples may appear in multiple test sets or never!

In [46]:
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

In [47]:
print(f"Total samples: {len(X)}")
print(f"Number of splits: 10")
print(f"Test size: 30%")
print(f"\nSplit details:")

for split_idx, (train_idx, test_idx) in enumerate(shuffle_split.split(X), 1):
    print(f"  Split {split_idx:2d}: Train={len(train_idx):3d} samples ({len(train_idx)/len(X)*100:.1f}%), "
          f"Test={len(test_idx):3d} samples ({len(test_idx)/len(X)*100:.1f}%)")


Total samples: 767
Number of splits: 10
Test size: 30%

Split details:
  Split  1: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  2: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  3: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  4: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  5: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  6: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  7: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  8: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split  9: Train=536 samples (69.9%), Test=231 samples (30.1%)
  Split 10: Train=536 samples (69.9%), Test=231 samples (30.1%)


In [48]:
shuffle_split = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

print(f"Total samples: {len(X)}")
print(f"Number of splits: 5")
print(f"Test size: 20%")
print(f"\nSplit details:")

Total samples: 767
Number of splits: 5
Test size: 20%

Split details:


In [49]:

for split_idx, (train_idx, test_idx) in enumerate(shuffle_split.split(X), 1):
    print(f"  Split {split_idx}: Train={len(train_idx)} samples, Test={len(test_idx)} samples")


  Split 1: Train=613 samples, Test=154 samples
  Split 2: Train=613 samples, Test=154 samples
  Split 3: Train=613 samples, Test=154 samples
  Split 4: Train=613 samples, Test=154 samples
  Split 5: Train=613 samples, Test=154 samples


## 4. Specialized techniques

### 4.1 Time Series Split

**What it does:** Respects temporal order - always trains on PAST data, tests on FUTURE data.

**When to use:**
- **Time series forecasting** (CRITICAL!)
- Any temporal data
- When order matters

**CRITICAL:** Never use regular K-Fold on time series - it will train on future data (data leakage)!

**How it works:** Training set grows with each split:
```
Split 1: Train[0:100] → Test[100:200]
Split 2: Train[0:200] → Test[200:300]
Split 3: Train[0:300] → Test[300:400]
```

In [50]:
# Create time-ordered data
X_time = X.copy()
y_time = y.copy()


In [51]:
tscv = TimeSeriesSplit(n_splits=5)

In [52]:
print(f"Total samples: {len(X_time)}")
print(f"Number of splits: 5")
print(f"\nSplit details (notice growing training set):")

for split_idx, (train_idx, test_idx) in enumerate(tscv.split(X_time), 1):
    print(f"\n  Split {split_idx}:")
    print(f"    Training: samples 0 to {len(train_idx)-1} ({len(train_idx)} total)")
    print(f"    Testing:  samples {len(train_idx)} to {len(train_idx)+len(test_idx)-1} ({len(test_idx)} total)")
    print(f"    Time order: Train on PAST [{train_idx[0]}-{train_idx[-1]}], "
          f"Test on FUTURE [{test_idx[0]}-{test_idx[-1]}]")

    

Total samples: 767
Number of splits: 5

Split details (notice growing training set):

  Split 1:
    Training: samples 0 to 131 (132 total)
    Testing:  samples 132 to 258 (127 total)
    Time order: Train on PAST [0-131], Test on FUTURE [132-258]

  Split 2:
    Training: samples 0 to 258 (259 total)
    Testing:  samples 259 to 385 (127 total)
    Time order: Train on PAST [0-258], Test on FUTURE [259-385]

  Split 3:
    Training: samples 0 to 385 (386 total)
    Testing:  samples 386 to 512 (127 total)
    Time order: Train on PAST [0-385], Test on FUTURE [386-512]

  Split 4:
    Training: samples 0 to 512 (513 total)
    Testing:  samples 513 to 639 (127 total)
    Time order: Train on PAST [0-512], Test on FUTURE [513-639]

  Split 5:
    Training: samples 0 to 639 (640 total)
    Testing:  samples 640 to 766 (127 total)
    Time order: Train on PAST [0-639], Test on FUTURE [640-766]


### 4.2 Group K-Fold Cross-Validation

**What it does:** Keeps all samples from the same group together in the same fold.

**When to use:**
- Medical data (multiple samples from same patient)
- User behavior (multiple actions from same user)
- School data (multiple students from same school)
- **Prevent group-level leakage**

**Example problem:** If Patient A has 5 blood tests, you don't want 3 tests in training and 2 in testing - that leaks information!

In [53]:
# Create groups (simulating students from different schools/groups)
# Using attendance rate quartiles as proxy for groups
groups = pd.qcut(df_clean['AttendanceRate'].values, q=4, labels=False, duplicates='drop')

print(f"\nDataset info:")
print(f"  Total samples: {len(X)}")
print(f"  Number of groups: {len(np.unique(groups))}")
print(f"\nSamples per group:")
for group_id in np.unique(groups):
    count = np.sum(groups == group_id)
    print(f"  Group {group_id}: {count} samples")


Dataset info:
  Total samples: 767
  Number of groups: 4

Samples per group:
  Group 0: 221 samples
  Group 1: 223 samples
  Group 2: 177 samples
  Group 3: 146 samples


In [55]:
group_kfold = GroupKFold(n_splits=4)

print(f"\nNumber of folds: 4")
print(f"\nSplit details:")

for fold_idx, (train_idx, test_idx) in enumerate(group_kfold.split(X, y, groups), 1):
    train_groups = np.unique(groups[train_idx])
    test_groups = np.unique(groups[test_idx])
    
    print(f"\n  Fold {fold_idx}:")
    print(f"    Training: {len(train_idx)} samples from groups {list(train_groups)}")
    print(f"    Testing:  {len(test_idx)} samples from groups {list(test_groups)}")
    print(f"    ✓ No overlap in groups between train and test!")


Number of folds: 4

Split details:

  Fold 1:
    Training: 544 samples from groups [0, 2, 3]
    Testing:  223 samples from groups [1]
    ✓ No overlap in groups between train and test!

  Fold 2:
    Training: 546 samples from groups [1, 2, 3]
    Testing:  221 samples from groups [0]
    ✓ No overlap in groups between train and test!

  Fold 3:
    Training: 590 samples from groups [0, 1, 3]
    Testing:  177 samples from groups [2]
    ✓ No overlap in groups between train and test!

  Fold 4:
    Training: 621 samples from groups [0, 1, 2]
    Testing:  146 samples from groups [3]
    ✓ No overlap in groups between train and test!


### 4.3 Stratified Group K-Fold

**What it does:** Combines stratification (class balance) with group separation.

**When to use:**
- Grouped data + Imbalanced classes
- Medical studies with patient groups
- Best of both worlds needed

**Example:** Hospital study with patients from different locations, and rare disease (imbalanced classes)

In [56]:

stratified_group_kfold = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=42)

print(f"\nNumber of folds: 4")
print(f"\nSplit details (maintaining both group integrity AND class balance):")

for fold_idx, (train_idx, test_idx) in enumerate(stratified_group_kfold.split(X, y, groups), 1):
    train_groups = np.unique(groups[train_idx])
    test_groups = np.unique(groups[test_idx])
    train_pass_rate = y[train_idx].mean() * 100
    test_pass_rate = y[test_idx].mean() * 100
    
    print(f"\n  Fold {fold_idx}:")
    print(f"    Training: {len(train_idx)} samples from groups {list(train_groups)}")
    print(f"              Pass Rate: {train_pass_rate:.1f}%")
    print(f"    Testing:  {len(test_idx)} samples from groups {list(test_groups)}")
    print(f"              Pass Rate: {test_pass_rate:.1f}%")
    print(f"    ✓ Groups separated AND class balance maintained!")



Number of folds: 4

Split details (maintaining both group integrity AND class balance):

  Fold 1:
    Training: 590 samples from groups [0, 1, 3]
              Pass Rate: 79.7%
    Testing:  177 samples from groups [2]
              Pass Rate: 81.9%
    ✓ Groups separated AND class balance maintained!

  Fold 2:
    Training: 546 samples from groups [1, 2, 3]
              Pass Rate: 80.2%
    Testing:  221 samples from groups [0]
              Pass Rate: 80.1%
    ✓ Groups separated AND class balance maintained!

  Fold 3:
    Training: 621 samples from groups [0, 1, 2]
              Pass Rate: 80.0%
    Testing:  146 samples from groups [3]
              Pass Rate: 80.8%
    ✓ Groups separated AND class balance maintained!

  Fold 4:
    Training: 544 samples from groups [0, 2, 3]
              Pass Rate: 80.9%
    Testing:  223 samples from groups [1]
              Pass Rate: 78.5%
    ✓ Groups separated AND class balance maintained!


## 5. Advanced Techniques

### 5.1 Stratified Shuffle Split

**What it does:** Combines Shuffle Split flexibility with stratification.

**When to use:**
- Imbalanced data + custom split sizes
- Need multiple random evaluations
- Want both stratification and flexibility

In [57]:

stratified_shuffle = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)


In [58]:
print(f"\nTotal samples: {len(X)}")
print(f"Number of splits: 10")
print(f"Test size: 25%")
print(f"\nSplit details (notice consistent pass rates):")

for split_idx, (train_idx, test_idx) in enumerate(stratified_shuffle.split(X, y), 1):
    train_pass_rate = y[train_idx].mean() * 100
    test_pass_rate = y[test_idx].mean() * 100
    
    print(f"  Split {split_idx:2d}: Train={len(train_idx)} samples (Pass: {train_pass_rate:.1f}%), "
          f"Test={len(test_idx)} samples (Pass: {test_pass_rate:.1f}%)")


Total samples: 767
Number of splits: 10
Test size: 25%

Split details (notice consistent pass rates):
  Split  1: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  2: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  3: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  4: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  5: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  6: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  7: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  8: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split  9: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)
  Split 10: Train=575 samples (Pass: 80.2%), Test=192 samples (Pass: 80.2%)


### 5.2 Predefined Split

**What it does:** Uses pre-assigned train/test split indicators.

**When to use:**
- Splits determined by business rules
- Specific organizational requirements
- Time-based splits (before/after date)
- Custom stratification needed

**Example use cases:**
- Train on 2023 data, test on 2024 data
- Train on Site A, test on Site B
- Custom business logic for splits

In [59]:
# Create predefined split based on attendance rate
# High attendance (>90) goes to test, rest goes to train
test_fold = np.where(df_clean['AttendanceRate'].values > 90, 0, -1)

print("\n📍 Example: Predefined Split Based on Attendance Rate")
print("-" * 80)
print("\nSplit Criteria: Students with >90% attendance → Test Set")
print("                Students with ≤90% attendance → Training Set")

predefined_split = PredefinedSplit(test_fold)

print(f"\nTotal samples: {len(X)}")


📍 Example: Predefined Split Based on Attendance Rate
--------------------------------------------------------------------------------

Split Criteria: Students with >90% attendance → Test Set
                Students with ≤90% attendance → Training Set

Total samples: 767


In [60]:

for split_idx, (train_idx, test_idx) in enumerate(predefined_split.split(), 1):
    train_attendance = df_clean['AttendanceRate'].iloc[train_idx].mean()
    test_attendance = df_clean['AttendanceRate'].iloc[test_idx].mean()
    
    print(f"\nSplit {split_idx}:")
    print(f"  Training Set:")
    print(f"    Size: {len(train_idx)} samples ({len(train_idx)/len(X)*100:.1f}%)")
    print(f"    Average Attendance: {train_attendance:.1f}%")
    print(f"    Pass Rate: {y[train_idx].mean()*100:.1f}%")
    
    print(f"\n  Testing Set:")
    print(f"    Size: {len(test_idx)} samples ({len(test_idx)/len(X)*100:.1f}%)")
    print(f"    Average Attendance: {test_attendance:.1f}%")
    print(f"    Pass Rate: {y[test_idx].mean()*100:.1f}%")

# Show split assignment
print(f"\nSplit Assignment Details:")
print(f"  Training samples (fold = -1): {np.sum(test_fold == -1)}")
print(f"  Testing samples (fold = 0):   {np.sum(test_fold == 0)}")



Split 1:
  Training Set:
    Size: 525 samples (68.4%)
    Average Attendance: 82.4%
    Pass Rate: 80.4%

  Testing Set:
    Size: 242 samples (31.6%)
    Average Attendance: 92.7%
    Pass Rate: 79.8%

Split Assignment Details:
  Training samples (fold = -1): 525
  Testing samples (fold = 0):   242
